In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 微調並部署基礎模型

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/language/tuning/tuning_text_bison.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在 Colab 中執行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/doggy8088/generative-ai/blob/main/language/tuning/tuning_text_bison.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在 GitHub 上檢視
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/blob/main/language/tuning/tuning_text_bison.zh.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在 Vertex AI Workbench 中開啟
    </a>
  </td>
</table>


| | |
|-|-|
|作者 | [Erwin Huizenga](https://github.com/erwinh85) |


建立大型語言模型需要大量的資料、大量的運算資源以及專門的技術。在 Vertex AI 上，微調可讓你自訂基礎模型以應付更特定的任務或知識領域。

提示設計雖然非常適合快速實驗，但若有訓練資料，你可以透過微調模型來提升品質。微調模型可讓你根據你希望模型執行的任務範例，來自訂模型回應。

如需了解微調的更多詳細資訊，請參閱 [官方文件](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-models)。


### 目標

本教學課程將教你如何針對新的未見過資料微調基礎模型，而且你將會使用下列 Google Cloud 產品：

- Vertex AI Generative AI Studio
- Vertex AI Pipelines
- Vertex AI Model Registry
- Vertex AI Endpoints

執行的步驟包括：

- 從 BQ 取得訓練資料並產生一個 JSONL 檔案
- 上傳訓練資料
- 建立一個管道作業
- 在 Vertex AI Model Registry 中檢查你的模型
- 從你的微調模型取得預測


### 配額
**重要事項** : 調整 text-bison@001 模型會使用 tpu-v3-8 訓練資源以及你 Google Cloud 專案隨附的配額。每個專案有八個 v3-8 核心配額，可容許執行同時執行一至兩個調整工作。如果你想執行更多同時執行工作，需要透過 [配額頁面](https://console.cloud.google.com/iam-admin/quotas) 要求額外配額。


### 費用
本教學指南使用 Google
Cloud 的計費元件：

* Vertex AI Generative AI Studio

了解 [Vertex AI 定價](https://cloud.google.com/vertex-ai/pricing)
，並使用 [定價計算器](https://cloud.google.com/products/calculator/)
，根據預計用量產生成本估算。


### 安裝 Vertex AI SDK


In [ ]:
!pip install google-cloud-aiplatform google-cloud-bigquery sequence-evaluate sentence-transformers rouge --upgrade --user

**僅 Colab：** 取消以下單元的註解以重新啟動Kernel或使用重新啟動按鈕。對於 Vertex AI Workbench，你可以使用頂端的按鈕重新啟動終端機。


In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### 驗證筆記本環境
* 如果你使用 **Colab** 執行此筆記本，取消註解下方的Cell並繼續。
* 如果你使用 **Vertex AI 工作台** ，請查看[此處](https://github.com/doggy8088/generative-ai/tree/main/setup-env)的設定說明。


In [1]:
from google.colab import auth

auth.authenticate_user()

### BigQuery IAM
現在你需要加入服務帳戶的權限：
- 前往主控台的 [IAM 頁面](https://console.cloud.google.com/iam-admin/)
- 尋找預設的運算服務帳戶。它應該看起來像這樣：`<project-number>-compute@developer.gserviceaccount.com`
- 分配 `bigquery.user` 給預設的運算服務帳戶


### 設定你的專案代號

**如果你不知道你的專案代號** , 你可能會使用 `gcloud` 來取得你的專案代號。否則，請查看支援網頁：[找出專案代號](https://support.google.com/googleapi/answer/7014113)。請在底下更新 `PROJECT_ID`。


In [ ]:
PROJECT_ID = "<your_project_id>"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

### 建立儲存區
現在你必須建立一個儲存區，我們將使用它來儲存調整資料。如需避免使用者在資源建立時發生重複，請為每個執行階段產生一個 UUID，並將其附加到此教學課程中建立的資源名稱上。


In [3]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

選擇儲存空間名稱並更新 `BUCKET_NAME` 參數。


In [4]:
BUCKET_NAME = "<your_bucket_uri>"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"
REGION = "us-central1"  # @param {type: "string"}

In [5]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "<your-bucket-name>":
    BUCKET_NAME = "vertex-" + UUID
    BUCKET_URI = f"gs://{BUCKET_NAME}"

僅在你的儲存空間尚不存在時：執行下列Cell來建立你的雲端儲存空間儲存空間。


In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

最後，透過檢查雲端儲存空間中的內容，驗證存取權限：


In [ ]:
! gsutil ls -al $BUCKET_URI

### 匯入函式庫


**Colab 只限** : 執行下列Cell以初始化 Vertex AI SDK。對於 Vertex AI Workbench，你不必執行這個動作。


In [8]:
import vertexai

vertexai.init(project=PROJECT_ID, location=REGION)

In [9]:
from typing import Union

import pandas as pd
from sklearn.model_selection import train_test_split

from google.cloud import aiplatform
from google.cloud import bigquery
from vertexai.language_models import TextGenerationModel

## 調整你的模型

現在你可以建立調整工作。透過使用 Generative AI Studio、cURL 或 Python SDK 建立管線工作來調整基礎模型。在本筆記本中，我們將使用 Python SDK。你將使用 Q&A 與 JSON 格式的脈絡資料集。

### 訓練資料
💾 你的模型調整資料集必須採用 JSONL 格式，其中每一行包含一個單一的訓練範例。你必須確保包含說明。

你將使用 BigQuery 公共資料集中的 StackOverflow 資料，限制為標籤為 `python` 的問題，並自 2020-01-01 以來接受回覆的答案。


首先建立一個輔助函式，讓你可以輕鬆的查詢 BigQuery 並且回傳結果作為 Pandas DataFrame。


In [10]:
def run_bq_query(sql: str) -> Union[str, pd.DataFrame]:
    """
    Run a BigQuery query and return the job ID or result as a DataFrame
    Args:
        sql: SQL query, as a string, to execute in BigQuery
    Returns:
        df: DataFrame of results from query,  or error, if any
    """

    bq_client = bigquery.Client(project=PROJECT_ID)

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    bq_client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql, job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    df = client_result.result().to_arrow().to_pandas()
    print(f"Finished job_id: {job_id}")

    return df

接下來定義查詢內容。


In [ ]:
df = run_bq_query(
    """SELECT
    CONCAT(q.title, q.body) as input_text,
    a.body AS output_text
FROM
    `bigquery-public-data.stackoverflow.posts_questions` q
JOIN
    `bigquery-public-data.stackoverflow.posts_answers` a
ON
    q.accepted_answer_id = a.id
WHERE
    q.accepted_answer_id IS NOT NULL AND
    REGEXP_CONTAINS(q.tags, "python") AND
    a.creation_date >= "2020-01-01"
LIMIT
    10000
"""
)

df.head()

應該有 10k 的問答資料。


In [ ]:
print(len(df))

讓我們將資料分成訓練和評估。對於抽取式問答任務我們建議使用 100+ 訓練範例。在這個情況中你將使用 800。


In [ ]:
# split is set to 80/20
train, evaluation = train_test_split(df, test_size=0.2)
evaluation = evaluation.sample(n=250, random_state=1)
print(len(train))
print(len(evaluation))

進行調校時，必須先將訓練資料轉換為 JSONL 格式。


In [ ]:
tune_jsonl = train.to_json(orient="records", lines=True)

print(f"Length: {len(tune_jsonl)}")
print(tune_jsonl[0:100])

接著，你可以在傳送至 Google Cloud Storage (GCS) 之前，先將它寫入本機 JSONL。


In [15]:
training_data_filename = "tune_data_stack_overflow_python_qa.jsonl"

with open(training_data_filename, "w") as f:
    f.write(tune_jsonl)

In [ ]:
tune_jsonl = evaluation.to_json(orient="records", lines=True)

print(f"Length: {len(tune_jsonl)}")
print(tune_jsonl[0:100])

In [17]:
evaluation_data_filename = "tune_eval_data_stack_overflow_python_qa.jsonl"

with open(evaluation_data_filename, "w") as f:
    f.write(tune_jsonl)

你接著可以將本機檔案匯出到 GCS，Vertex AI 便可於調整作業中使用該檔案。


In [ ]:
! gsutil cp $training_data_filename $evaluation_data_filename $BUCKET_URI

你可以檢查檔案是否順利傳輸到你的 Google Cloud Storage 儲存空間：


In [ ]:
! gsutil ls -al $BUCKET_URI

In [20]:
TRAINING_DATA_URI = f"{BUCKET_URI}/{training_data_filename}"
EVAUATION_DATA_URI = f"{BUCKET_URI}/{evaluation_data_filename}"

### 模型微調
現在是開始微調模型的時候了。你將使用 Vertex AI SDK 提交微調工作。

#### 建議的微調設定
✅ 以下是根據任務微調基礎模型的一些建議設定，例如此範例中的問答。你可以在 [文件](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-models) 中找到更多資訊。

萃取式問答：
- 確保訓練資料集大小超過 100 筆
- 訓練步驟 [100-500]。你可以嘗試多個值以取得特定資料集的最佳效能 (例如 100、200、500)


In [ ]:
# create tensorboard
display_name = "Adapter tuning - "

tensorboard = aiplatform.Tensorboard.create(
    display_name=display_name,
    project=PROJECT_ID,
    location=REGION,
)

print(tensorboard.display_name)
print(tensorboard.resource_name)

In [ ]:
# Get tensorboard_id thats used in the pipeline
tensorboard_id = tensorboard.resource_name.split("tensorboards/")[-1]
print(tensorboard_id)

In [30]:
MODEL_NAME = f"genai-workshop-tuned-model-{UUID}"
TRAINING_STEPS = 100

In [35]:
pipeline_arguments = {
    "model_display_name": MODEL_NAME,
    "location": REGION,
    "large_model_reference": "text-bison@001",
    "project": PROJECT_ID,
    "train_steps": TRAINING_STEPS,
    "dataset_uri": TRAINING_DATA_URI,
    "evaluation_interval": 20,
    "evaluation_data_uri": EVAUATION_DATA_URI,
    "tensorboard_resource_id": tensorboard_id,
}

pipeline_root = f"{BUCKET_URI}/{MODEL_NAME}"
template_path = "https://us-kfp.pkg.dev/ml-pipeline/large-language-model-pipelines/tune-large-model/v2.0.0"

In [55]:
# Function that starts the tuning job
def tuned_model(
    project_id: str,
    location: str,
    template_path: str,
    model_display_name: str,
    pipeline_arguments: str,
):
    """Prompt-tune a new model, based on a prompt-response data.

    "training_data" can be either the GCS URI of a file formatted in JSONL format
    (for example: training_data=f'gs://{bucket}/{filename}.jsonl'), or a pandas
    DataFrame. Each training example should be JSONL record with two keys, for
    example:
      {
        "input_text": <input prompt>,
        "output_text": <associated output>
      },

    Args:
      project_id: GCP Project ID, used to initialize aiplatform
      location: GCP Region, used to initialize aiplatform
      template_path: path to the template
      model_display_name: Name for your model.
      pipeline_arguments: arguments used during pipeline runtime
    """

    aiplatform.init(project=project_id, location=location)

    from google.cloud.aiplatform import PipelineJob

    job = PipelineJob(
        template_path=template_path,
        display_name=model_display_name,
        parameter_values=pipeline_arguments,
        location=REGION,
        pipeline_root=pipeline_root,
        enable_caching=True,
    )

    return job

接下來，是時候開始你的微調工作了。

**免責聲明：** 微調和部署模型需要時間。


In [ ]:
job = tuned_model(PROJECT_ID, REGION, template_path, MODEL_NAME, pipeline_arguments)

In [ ]:
job.submit()

按上述連結，你可以查看你的串接執行。如以下截圖所示，它會執行以下步驟：

- 驗證
- 匯出管理的資料集
- 將 JSONL 轉換為 TFRecord
- 大型語言模型微調
- 上傳 LLM 模型


`job.state` 讓你可以查看管線的狀態。


In [ ]:
job.state

## 在 Vertex AI Model Registry 上檢視你調整完成的基礎模型
調整工作完成後，你的模型會顯示在 Vertex AI Model Registry 上。以下 Python SDK 範例會顯示如何列出經過調整的模型。


In [ ]:
def list_tuned_models(project_id, location):
    aiplatform.init(project=project_id, location=location)
    model = TextGenerationModel.from_pretrained("text-bison@001")
    tuned_model_names = model.list_tuned_model_names()
    print(tuned_model_names)

In [ ]:
list_tuned_models(PROJECT_ID, REGION)

你也可以使用 Google Cloud Console UI 檢視 [Vertex AI Model Registry](https://console.cloud.google.com/vertex-ai/models) 中的所有模型。下面你可以看到 Vertex AI Model Registry 中可用的已調整基礎模型範例。


## 使用已調校模型取得預測
現在是取得預測的時候了。首先，你需要從 Vertex AI 模型註冊取得最新的調校模型。


In [ ]:
def fetch_model(project_id, location):
    aiplatform.init(project=project_id, location=location)
    model = TextGenerationModel.from_pretrained("text-bison@001")
    list_tuned_models = model.list_tuned_model_names()
    tuned_model = list_tuned_models[0]

    return tuned_model

In [ ]:
deployed_model = fetch_model(PROJECT_ID, REGION)
deployed_model = TextGenerationModel.get_tuned_model(deployed_model)

現在你可以開始傳送提示到 API。請隨時更新以下提示。


In [ ]:
PROMPT = """
How can I store my TensorFlow checkpoint on Google Cloud Storage?

Python example:

"""

In [ ]:
print(deployed_model.predict(PROMPT))

## 評估
對模型執行評估來了解其效能至關重要。評估可使用 F1 或 Rouge 等評估指標以自動化方式進行。你也可以運用人工評估方法。人工評估方法涉及要求人員評分 LLM 回答的品質。這可透過群眾外包或由專家評估回應來執行。一些標準的人工評估指標包括流暢度、連貫性、相關性，和資訊性。你通常會想要選擇多種評估指標，以便充分了解模型效能。以下會提供評估執行的範例。

在此範例中，你將使用 [sequence-evaluate](https://pypi.org/project/sequence-evaluate/) 來評估調整後的模型。


In [ ]:
from seq_eval import SeqEval

evaluator = SeqEval()

在筆記本前面，你建立了一個訓練和評量資料集。現在是取得部分評量資料的時候了。你將使用這些問題從我們微調的模型取得回應，而我們會使用答案作為參考：

- **候選項目：** 微調模型產生的答案。
- **參考：** 我們將用於比較的原始答案。


In [ ]:
evaluation = evaluation.head(10)  # you can change the number of rows you want to use
evaluation_question = evaluation["input_text"]
evaluation_answer = evaluation["output_text"]

現在你可以使用根據你從評估資料集中提出的問題調整的模型繼續生成候選人。


In [ ]:
candidates = []

for i in evaluation_question:
    response = deployed_model.predict(i)
    candidates.append(response.text)

len(candidates)

你還需建立我們的參考清單。這些將會用於評估模型的績效。


In [ ]:
references = evaluation_answer.tolist()

len(references)

接下來你會產生評估指標。 `evaluator.evaluate` 將會傳回幾個評估指標。其中一些重要的指標有：
- [Blue](https://en.wikipedia.org/wiki/BLEU)：BLEU 評估指標是機器產生的文字和人工撰寫的參考文字的相似度指標。
- [Rouge](https://en.wikipedia.org/wiki/ROUGE_(metric))：ROUGE 評估指標是機器產生的文字和人工撰寫的參考文字的重疊度指標。


In [ ]:
scores = evaluator.evaluate(candidates, references, verbose=False)
print(scores)